In [1]:
#Importamos las modulos
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import time
from numpy import linalg as LA
import random
from time import time
%matplotlib qt

In [2]:
#Definimos la funcion a minimizar
def f(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [3]:
#Definir funcion para calcular la matriz del gradiente
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(f([x[0]+delta,x[1]])- f([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(f([x[0],x[1]+delta])- f([x[0],x[1]-delta]))/(2*delta)
    return grad

In [4]:
#Utilizamos el algoritmo de golden section para encontrar alpha optimo
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = f(x+alpha1*search);
    falpha2 = f(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = f(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = f(x+alpha1*search);
        if np.abs(f(x+alpha1*search)- f(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [5]:
#Graficamos la funcion de prueba
X1 = np.arange(-5, 5, 0.01)
X2 = np.arange(-5, 5, 0.01)
x1, x2 = np.meshgrid(X1, X2)
z = f([x1,x2])
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
surf = ax.plot_surface(x1, x2, z, cmap=cm.autumn, linewidth=0, antialiased = False, alpha=0.2)
plt.show()

In [6]:
#Variable en comun de los métodos
delta = 1e-3 
ep1 = 1e-3

In [7]:
puntosBFP = []

In [8]:
def graficaUnaaUna(puntos, nombreMetodo):
    #Generamos valores ramdom para generar un color diferente de grafica
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    aux = 0
    x = np.linspace(-1.3, 1.3, 100)
    y = np.linspace(-3, 1.3, 100)
    xx, yy = np.meshgrid(x, y)
    z = f([xx, yy])
    plt.contour(x,y,z,20)
    plt.plot(-1,1, 'ro--',  markersize=6)
    plt.plot(0.5,0, 'go--', markersize=6)
    #Graficamos los trazos (direcciones) que va generando el algoritmo
    for pts in puntos:
        if(aux==len(puntos)-1):
            plt.plot(pts[0],pts[1], c=color, linewidth=2, label=str(nombreMetodo))
        else:
            plt.plot(pts[0],pts[1], c=color, linewidth=2)
        aux = aux + 1
    plt.legend()    
    plt.show

## DFP method

In [9]:
start_time = time()
xi = [-1,1]
x = xi
Bi = np.eye(len(x))
fx_prev = f(x)
print('Valor de la función inicial = %f ' % fx_prev) #Valor inicial de la funcion con el punto inicial
iteraciones = 1 #Conteo de numero de iteraciones
Gx = gradient(x,delta) #Direccion de busqueda
S = -Gx 
alpha,fx_prev = golden(x,S,xi,ep1)
xi_1 = x + alpha*S       
for j in range(1000):
    deltax=xi_1-x
    Gxi=gradient(xi_1,delta)
    Gi = Gxi-Gx
    Bi = Bi + (np.matmul(np.reshape(deltax,(2,1)),np.reshape(deltax,(1,2)))/np.matmul(deltax,Gi.transpose())) - (np.matmul(np.matmul(np.matmul(Bi, np.reshape(Gi,(2,1))), np.reshape(Gi,(1,2))),Bi) / np.matmul(np.matmul(np.reshape(Gi,(1,2)),Bi), np.reshape(Gi,(2,1))))
    si = np.matmul(-Bi,Gxi.transpose())
    si = np.ndarray.flatten(si.transpose())
    alpha,fx_curr = golden(xi_1[:],si,xi,ep1)
    print('{0}\t[{1:.3f},{2:.3f}]\t{3:.3f}\t{4:.3f}'.format(j, x[0], x[1],fx_curr,LA.norm(Gx)))
    if abs(fx_curr-fx_prev)<ep1 or LA.norm(Gx)<ep1:
        break
    fx_prev=fx_curr
    Gx = Gxi
    iteraciones+=1
    puntosBFP.append([[x[0],xi_1[0]],[x[1],xi_1[1]]])
    x=xi_1
    xi_1 = x + alpha*si
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
graficaUnaaUna(puntosBFP, 'BFP')  

Valor de la función inicial = 270.294169 
0	[-1.000,1.000]	-0.864	444.316
1	[-0.389,-0.009]	-4.201	33.677
2	[0.022,0.199]	-6.867	40.221
3	[0.200,0.168]	-7.267	30.562
4	[0.364,0.068]	-7.349	7.573
5	[0.475,0.031]	-7.352	6.891
6	[0.490,0.007]	-7.352	0.861
Tiempo transcurrido: 4.982948
Iteraciones: 7
